<a href="https://colab.research.google.com/github/pranav2902/scale_ai_project/blob/main/pranav_scale_ai_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
import tensorflow as tf
from tensorflow import keras

In [ ]:
batch_size = 256  # Batch size for training.
epochs = 20  # Number of epochs to train for.
latent_dim = 256  # Latent dimensionality of the encoding space.
data_path = "/content/drive/MyDrive/polynomial/train.txt"

In [ ]:
# Vectorize the data.
input_texts = []
target_texts = []
input_characters = set()
target_characters = set()
with open(data_path, "r", encoding="utf-8") as f:
    lines = f.read().split("\n")
for line in lines[: len(lines) - 1]:
    input_text, target_text = line.split("=")
    # We use "=" as the "start sequence" character
    # for the targets, and "\n" as "end sequence" character.
    target_text = "=" + target_text + "\n"
    input_texts.append(input_text)
    target_texts.append(target_text)
    for char in input_text:
        if char not in input_characters:
            input_characters.add(char)
    for char in target_text:
        if char not in target_characters:
            target_characters.add(char)

input_characters = sorted(list(input_characters))
target_characters = sorted(list(target_characters))
num_encoder_tokens = len(input_characters)
num_decoder_tokens = len(target_characters)
max_encoder_seq_length = max([len(txt) for txt in input_texts])
max_decoder_seq_length = max([len(txt) for txt in target_texts])

print("Number of samples:", len(input_texts))
print("Number of unique input tokens:", num_encoder_tokens)
print("Number of unique output tokens:", num_decoder_tokens)
print("Max sequence length for inputs:", max_encoder_seq_length)
print("Max sequence length for outputs:", max_decoder_seq_length)

input_token_index = dict([(char, i) for i, char in enumerate(input_characters)])
target_token_index = dict([(char, i) for i, char in enumerate(target_characters)])

encoder_input_data = np.zeros(
    (len(input_texts), max_encoder_seq_length, num_encoder_tokens), dtype="float32"
)
decoder_input_data = np.zeros(
    (len(input_texts), max_decoder_seq_length, num_decoder_tokens), dtype="float32"
)
decoder_target_data = np.zeros(
    (len(input_texts), max_decoder_seq_length, num_decoder_tokens), dtype="float32"
)
print(input_token_index)
for i, (input_text, target_text) in enumerate(zip(input_texts, target_texts)):
    for t, char in enumerate(input_text):
        encoder_input_data[i, t, input_token_index[char]] = 1.0
    #encoder_input_data[i, t + 1 :, input_token_index[" "]] = 1.0
    for t, char in enumerate(target_text):
        # decoder_target_data is ahead of decoder_input_data by one timestep
        decoder_input_data[i, t, target_token_index[char]] = 1.0
        if t > 0:
            # decoder_target_data will be ahead by one timestep
            # and will not include the start character.
            decoder_target_data[i, t - 1, target_token_index[char]] = 1.0
    #decoder_input_data[i, t + 1 :, target_token_index[" "]] = 1.0
    #decoder_target_data[i, t:, target_token_index[" "]] = 1.0

Number of samples: 1000000
Number of unique input tokens: 28
Number of unique output tokens: 30
Max sequence length for inputs: 29
Max sequence length for outputs: 30
{'(': 0, ')': 1, '*': 2, '+': 3, '-': 4, '0': 5, '1': 6, '2': 7, '3': 8, '4': 9, '5': 10, '6': 11, '7': 12, '8': 13, '9': 14, 'a': 15, 'c': 16, 'h': 17, 'i': 18, 'j': 19, 'k': 20, 'n': 21, 'o': 22, 's': 23, 't': 24, 'x': 25, 'y': 26, 'z': 27}


In [ ]:
#Define an input sequence and process it.
encoder_inputs = keras.Input(shape=(None, num_encoder_tokens))
encoder = keras.layers.LSTM(latent_dim, return_state=True,dropout = 0.1)
encoder_outputs, state_h, state_c = encoder(encoder_inputs)

# We discard `encoder_outputs` and only keep the states.
encoder_states = [state_h, state_c]

# Set up the decoder, using `encoder_states` as initial state.
decoder_inputs = keras.Input(shape=(None, num_decoder_tokens))

# We set up our decoder to return full output sequences,
# and to return internal states as well. We don't use the
# return states in the training model, but we will use them in inference.
decoder_lstm = keras.layers.LSTM(latent_dim, return_sequences=True, return_state=True, dropout = 0.5)
decoder_outputs, _, _ = decoder_lstm(decoder_inputs, initial_state=encoder_states)
decoder_dense = keras.layers.Dense(num_decoder_tokens, activation="softmax")
decoder_outputs = decoder_dense(decoder_outputs)

# Define the model that will turn
# `encoder_input_data` & `decoder_input_data` into `decoder_target_data`
model = keras.Model([encoder_inputs, decoder_inputs], decoder_outputs)

In [ ]:
model.compile(
    optimizer="rmsprop", loss="categorical_crossentropy", metrics=["accuracy"]
)
# Tried rmsprop before, try adam next

model.fit(
    [encoder_input_data, decoder_input_data],
    decoder_target_data,
    batch_size=batch_size,
    epochs=epochs,
    validation_split=0.2,
)
# Save model
model.save("model_pranav_20ep_modularized_dropout")

Epoch 1/20
3125/3125 [==============================] - 903s 288ms/step - loss: 0.8529 - accuracy: 0.3410 - val_loss: 0.5793 - val_accuracy: 0.8278
Epoch 2/20
3125/3125 [==============================] - 1066s 341ms/step - loss: 0.5646 - accuracy: 0.8418 - val_loss: 0.4857 - val_accuracy: 0.8737
Epoch 3/20
3125/3125 [==============================] - 936s 299ms/step - loss: 0.4943 - accuracy: 0.8679 - val_loss: 0.4216 - val_accuracy: 0.8901
Epoch 4/20
3125/3125 [==============================] - 876s 280ms/step - loss: 0.4638 - accuracy: 0.8749 - val_loss: 0.3994 - val_accuracy: 0.8943
Epoch 5/20
3125/3125 [==============================] - 886s 284ms/step - loss: 0.4459 - accuracy: 0.8798 - val_loss: 0.3887 - val_accuracy: 0.8978
Epoch 6/20
3125/3125 [==============================] - 881s 282ms/step - loss: 0.4304 - accuracy: 0.8849 - val_loss: 0.3794 - val_accuracy: 0.9045
Epoch 7/20
3125/3125 [==============================] - 889s 284ms/step - loss: 0.4183 - accuracy: 0.8896 - val

INFO:tensorflow:Assets written to: model_pranav_20ep_modularized_dropout/assets


INFO:tensorflow:Assets written to: model_pranav_20ep_modularized_dropout/assets


In [ ]:
!zip -r /content/model_pranav_20ep_modularized_dropout.zip /content/model_pranav_20ep_modularized_dropout


  adding: content/model_pranav_20ep_modularized_dropout/ (stored 0%)
  adding: content/model_pranav_20ep_modularized_dropout/variables/ (stored 0%)
  adding: content/model_pranav_20ep_modularized_dropout/variables/variables.index (deflated 62%)
  adding: content/model_pranav_20ep_modularized_dropout/variables/variables.data-00000-of-00001 (deflated 7%)
  adding: content/model_pranav_20ep_modularized_dropout/keras_metadata.pb (deflated 90%)
  adding: content/model_pranav_20ep_modularized_dropout/saved_model.pb (deflated 91%)
  adding: content/model_pranav_20ep_modularized_dropout/assets/ (stored 0%)


In [ ]:
!unzip /content/model_pranav_20ep_modularized_dropout.zip -d /content/model_pranav_20ep_modularized_dropout


Archive:  /content/model_pranav_20ep_modularized_dropout.zip
   creating: /content/model_pranav_20ep_modularized_dropout/content/model_pranav_20ep_modularized_dropout/
   creating: /content/model_pranav_20ep_modularized_dropout/content/model_pranav_20ep_modularized_dropout/variables/
  inflating: /content/model_pranav_20ep_modularized_dropout/content/model_pranav_20ep_modularized_dropout/variables/variables.index  
  inflating: /content/model_pranav_20ep_modularized_dropout/content/model_pranav_20ep_modularized_dropout/variables/variables.data-00000-of-00001  
  inflating: /content/model_pranav_20ep_modularized_dropout/content/model_pranav_20ep_modularized_dropout/keras_metadata.pb  
  inflating: /content/model_pranav_20ep_modularized_dropout/content/model_pranav_20ep_modularized_dropout/saved_model.pb  
   creating: /content/model_pranav_20ep_modularized_dropout/content/model_pranav_20ep_modularized_dropout/assets/


In [ ]:
model = tf.keras.models.load_model('/content/model_pranav_20ep_modularized_dropout/content/model_pranav_20ep_modularized_dropout')

In [ ]:
model.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, None, 28)]   0           []                               
                                                                                                  
 input_2 (InputLayer)           [(None, None, 30)]   0           []                               
                                                                                                  
 lstm (LSTM)                    [(None, 256),        291840      ['input_1[0][0]']                
                                 (None, 256),                                                     
                                 (None, 256)]                                                     
                                                                                              

In [ ]:
# Saving model weights
weights_file = open("weights.txt", "w+")
for layer in model.layers:
    weights_file.write(str(layer.get_weights()))
    weights_file.write("\n")
weights_file.close()

In [ ]:
!zip -r /content/MySubmission.zip /content/MySubmission


  adding: content/MySubmission/ (stored 0%)
  adding: content/MySubmission/.ipynb_checkpoints/ (stored 0%)
  adding: content/MySubmission/network.txt (deflated 84%)
  adding: content/MySubmission/weights.txt (deflated 64%)
  adding: content/MySubmission/requirements.txt (stored 0%)
